Inspecting and Explaining Models
===

Author: Nathan A. Mahynski

Date: 2023/09/12

Description: How can we inspect models and explain their predictions?

<h3>Load the Data</h3>

Model Performance
---

Authenticators have different metrics that determine their performance.  

SIMCA table of predictions is like CM,

CM help explain how discriminative models perform and where they tend to go awry.  This can be important to understand for diagonize lack of data, or to possibly to consider re-classifying the input.

In [ ]:
confusion matrix from display
ROC

Global Explanation  Methods
---

These explain the model in general terms, like how much it relies on certain features on average to make predictions.

In [ ]:
PFI

In [ ]:
PDP

In [ ]:
SHAP - squashing function

Local Explanation Methods
---

These help us explain a single prediction.

In [ ]:
SHAP, ICE

Learning Curves
---

In [ ]:
learning curve from display